<center><img src="chunking.png" width="200"></center>

# CV Chunking PDF Docuemnts, Load in dataabse Prompt RAG question 
sono configurati diversi metodi di chunking riferiemnto 
https://towardsdatascience.com/how-to-chunk-text-data-a-comparative-analysis-3858c4a0997a

* Spacy Sentence Splitter 
* Langchain Character Text Splitter  
* NLTK Sentence Tokenizer 

Pacchetti da installare 

In [ ]:
pip install PyPDF2

In [ ]:
pip install nltk

In [ ]:
pip install pydantic

In [ ]:
pip install langchain

In [ ]:
pip install langchain==0.0.349

In [ ]:
pip install spacy


In [ ]:
!python3 -m spacy download en_core_web_sm

In [ ]:
pip list | grep langchain


In [ ]:
pip install  sentence_transformers

In [ ]:
pip install sklearn.cluster

In [ ]:
pip install oracledb

In [ ]:
pip install /Users/LBINDI/Downloads/p36050269_230000_Linux-x86-64/drivers/oracledb-2.0.0.dev20231121-cp311-cp311-macosx_10_9_universal2.whl

Fine pacchetti da installare

Python librerie Enviroment

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import spacy
from PyPDF2 import PdfReader
import nltk
from langchain.text_splitter import CharacterTextSplitter
import oracledb
import cohere
import os
import time



Cartella con pdf dei CV


In [2]:
cartella = '/Users/LBINDI/GreenHouse/Documents/LB/Cloud/LAB_ORDS/Embedding/docs/CV_PDF'

Cohere API KEY

In [3]:
CO_API_KEY="kKC103gzFwcLvsGfSNeXZsuN38BHrBAelzgefAez"
co = cohere.Client(CO_API_KEY)

Oracle enviroment 

In [102]:
# Connect to your Oracle 23.4 database
connection = oracledb.connect(
    user="vector",
    password="vector",
    dsn="158.101.175.18:1521/freepdb1"
    )
cursor = connection.cursor()


Drop and create table Vector_CV_PDF

In [ ]:
cursor.execute("""
    begin
        execute immediate 'drop table Vector_CV_PDF';
        exception when others then if sqlcode <> -942 then raise; end if;
    end;""")
# Create table Vector_TEST1
cursor.execute("""
    create table Vector_CV_PDF (
        id_pdf number,
        id_chunk number,
        nome_file varchar2(300),       
        v vector(1024, float32),
        chunk varchar2(4000),
        primary key (id_pdf,id_chunk))""")

#v vector(384, float32) --> 384 e' la dimensione

Funzione per estrarre i file PDF  all'interno della cartella fornita

In [5]:
def elenca_files_con_percorsi(carta):
    lista_files = []
    for cartella_radice, sottocartelle, files in os.walk(carta):
        for file in files:
            if file.lower().endswith('.pdf'):  # Verifica se il file ha estensione PDF
                  percorso_completo = os.path.join(cartella_radice, file)
                  lista_files.append(percorso_completo)
    return lista_files
#print (len(lista_file))
#print (lista_file)


Funzione per convertire pdf to testo

In [6]:

# Extracting Text from PDF
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf = PdfReader(file)
        text = " ".join(page.extract_text() for page in pdf.pages)
        
    return text



# Spacy
Funzione per suddivisione in chunk mediante spacy del testo , tennde a creare piccoli chiuck in confronto con  Langchain Character Text Splitter, . Puo essere vantaggioso con piccoli testi 

In [7]:
# chuk Text using spacy 
def divide_in_frase(testo):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(testo)
    frasi = list(doc.sents)
    return frasi
#testo_da_dividere = "Questo è un esempio di testo. Può contenere più di una frase. SpaCy è uno strumento di elaborazione del linguaggio naturale."
#frasi_divise = divide_in_frase(testo_da_dividere)


# Langchain Character Text Splitter
RecursiveCharacterTextSplitter ricorsivamente divide il testo sulla base di specifici caratteri .
Puo essere vantaggioso con Testi generici

In [8]:
def custom_text_splitter(text, chunk_size=100, chunk_overlap=10):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    
    chunks = splitter.create_documents([text])
    return chunks




# NLTK Sentence Tokenizer
Il Natural Language Toolkit (NLTK) fornisce una funzione utile per suddividere il testo in frasi. e' un modo semplice ed efficiente per dividere un ampio corpo di testo in singole frasi,ma ha alcune limitazioni:
* Dipendenza dalla lingua:Funziona bene con l'inglese ma potrebbe non fornire risultati accurati con altre lingue senza una configurazione aggiuntiva.
* Abbreviazioni e punteggiatura: il tokenizzatore può occasionalmente interpretare erroneamente le abbreviazioni o altri segni di punteggiatura.
* Mancanza di comprensione semantica: come la maggior parte dei tokenizzatori, il tokenizzatore di frasi NLTK non considera la relazione semantica tra le frasi. Pertanto, un contesto che si estende su più frasi potrebbe andare perso nel processo di tokenizzazione.

* nltk.punkt è un modulo di NLTK (Natural Language Toolkit) che fornisce un tokenizer di frasi. Il tokenizer divide un testo in una lista di frasi utilizzando un algoritmo non supervisionato per costruire un modello per le parole di abbreviazione, le collocazioni e le parole che iniziano le frasi.
* nltk.corpus: fornisce accesso a una vasta gamma di corpora etichettati e non etichettati, come il corpus Brown, il corpus Penn Treebank, il corpus WordNet e molti altri.
* nltk.tokenize: fornisce una serie di tokenizzatori per la suddivisione del testo in parole, frasi e altre unità.
* nltk.stem: fornisce una serie di algoritmi di stemming per ridurre le parole alle loro forme di base.
* nltk.tag: fornisce una serie di modelli di tagging per l’etichettatura delle parti del discorso, l’etichettatura delle entità denominate e altro ancora.
* nltk.chunk: fornisce una serie di modelli di chunking per l’identificazione di frasi nominali, verbi e altre unità sintattiche.
* nltk.parse: fornisce una serie di parser per l’analisi sintattica del testo.
* nltk.sentiment: fornisce una serie di modelli per l’analisi del sentiment.

In [ ]:
nltk.download('punkt')

def split_text_into_sentences(text):
    sentences = nltk.sent_tokenize(text)
    return sentences

# Confronta i diversi metodi 

In [16]:
id_file=0
lista_file= elenca_files_con_percorsi(cartella)
for nume_file in range(len(lista_file)):
    id_file+=1
    id_chunk=0
    # print (id_nome_file)
    # Extract text from the PDF and split it into sentences
    nome_file=lista_file[nume_file]
    text = extract_text_from_pdf(lista_file[nume_file])
    #print (text)
    spacy_frasi_divise = divide_in_frase(text)
    splitter_result_chunks = custom_text_splitter(text)
    nltk_sentences = split_text_into_sentences(text)
    print ('file : ', nume_file)
    print ('Spacy Sentence Splitter num chunk ',len(spacy_frasi_divise))
    print ('* Langchain Character Text Splitter num chunk ',len(splitter_result_chunks))
    print ('NLTK Sentence Tokenizer num chunk ',len(nltk_sentences))

   # print (spacy_frasi_divise)
   # print (splitter_result_chunks)
   # print (splitter_result_chunks[0].page_content)
   # print (nltk_sentences)


file :  0
Spacy Sentence Splitter num chunk  17
* Langchain Character Text Splitter num chunk  15
NLTK Sentence Tokenizer num chunk  15
['First Name: Luca Last Name: Migliori Address: Mecenati Street, Florence 00302 Profile: Machine Learning expert with a strong background in designing and implementing advanced models.', 'Specialized in Python, TensorFlow, and PyTorch, I take a practical approach to data analysis and solving complex problems.', 'Experience: University researcher, where I optimized existing models and integrated machine learning solutions into business processes.', 'Previously worked at the university, focusing on specific algorithms for the military sector.', 'Key Skills: Excellent proficiency in Python, TensorFlow, and PyTorch for developing machine learning models.', 'Established experience in regression, classification, and clustering algorithms.', 'Advanced skills in NLP with BERT and GPT.', "Education: Master's degree in Computer Engineering from the University of

 # Implementazione
 Conversione  di tutti i pdf, della cartella fornita, in text, successivamente suddiviso il testo  in chuck mediante spacy e traformati in vector embedding con Cohere embed , alla fine inseriti all'interno della tabella vector_PDF del database Oracel

In [19]:
id_file=0
cursor.setinputsizes(None,None,oracledb.DB_TYPE_VARCHAR, oracledb.DB_TYPE_VECTOR )
cohere_model_def='embed-english-v3.0'
cohere_input_type='search_document'
lista_file= elenca_files_con_percorsi(cartella)
for nume_file in range(len(lista_file)):
    id_file+=1
    id_chunk=0
    # print (id_nome_file)
    # Extract text from the PDF and split it into sentences
    nome_file=lista_file[nume_file]
    text = extract_text_from_pdf(lista_file[nume_file])
    #print (text)
    #scegliere quale metodo di suddivisione testo utilizzare
    #
    #frasi_divise = spacy_frasi_divise = divide_in_frase(text)
    #frasi_divise = splitter_result_chunks = custom_text_splitter(text)
    frasi_divise = nltk_sentences = split_text_into_sentences(text)
    #
    api_call=0
    for frase in frasi_divise:
        api_call+=1
        if api_call<90:
            id_chunk+=1
            response = co.embed(
                    #spacy_frasi_divise
                    #texts=[frase.text],
                    #nltk_sentences
                    texts=[frase],
                    #splitter_result_chunks
                    #texts=[frase non provato non frase[0].page_content
                    model=cohere_model_def,
                    input_type=cohere_input_type
            )
            #print (response)
            vector_result = response.embeddings[0]
            #print (vector_result)
            ##print(id_file,nome_file,vector_result,frase)
            # Insert the vector using the bind variable from the generated vector
            cursor.execute("insert into Vector_CV_PDF values (:1,:2,:3,:4,:5)", [id_file,id_chunk,nome_file, vector_result,frase])
        else :
            api_call=0
            time.sleep(60)
    print ( 'numero di chunk per il pdf  : ' , nome_file, ' -- ',id_chunk)
    connection.commit()

print ( 'numero documenti pdf totali inseriti : ' , id_file)




numero di chunk per il pdf  :  /Users/LBINDI/GreenHouse/Documents/LB/Cloud/LAB_ORDS/Embedding/docs/CV_PDF/luca_Migliori.pdf  --  15
numero di chunk per il pdf  :  /Users/LBINDI/GreenHouse/Documents/LB/Cloud/LAB_ORDS/Embedding/docs/CV_PDF/pino_sarsi.pdf  --  16
numero di chunk per il pdf  :  /Users/LBINDI/GreenHouse/Documents/LB/Cloud/LAB_ORDS/Embedding/docs/CV_PDF/Paolo_checchi.pdf  --  14
numero di chunk per il pdf  :  /Users/LBINDI/GreenHouse/Documents/LB/Cloud/LAB_ORDS/Embedding/docs/CV_PDF/eugenio_bernaci.pdf  --  16
numero di chunk per il pdf  :  /Users/LBINDI/GreenHouse/Documents/LB/Cloud/LAB_ORDS/Embedding/docs/CV_PDF/mario_bianchi.pdf  --  13
numero documenti pdf totali inseriti :  5


Verifica contenuto tabella

In [ ]:
# Retrieve the vector
cursor.execute('select * from  Vector_CV_PDF')
#cursor.execute('select max(id_pdf) from  Vector_PDF')
for row in cursor:
     print(row)


Richiesta al modello e conversione in embedding
modificare la domanda a proprio piacimento

In [ ]:
prompt_question = "I've opened a new position for an Italian coding expert in Taleo application , in the meantime could you providing internal name showing me the highlights of this person"
response = co.embed(
        texts=[prompt_question],
        model='embed-english-v3.0',
        input_type='search_query'
)
print(response)

Ricerca dei chunk nel vector db con il vettore KNN piu vicino 

In [121]:
import array
from typing import List
vector_prompt:List[float]=response.embeddings[0]

cursor = connection.cursor()
sql = """SELECT CHUNK  from Vector_CV_PDF where (ID_PDF, id_chunk) in 
(
select ID_PDF ,substr(ranking,instr(ranking,'-',-1)+1)chunk_id from (
select ID_PDF , min(ranking) ranking
  from (
        select ID_PDF,VECTOR_DISTANCE(V,:1)||'--'||id_chunk ranking 
          from Vector_CV_PDF  
        order by VECTOR_DISTANCE(V,:2)
        )
group by id_pdf order by 2,1 desc FETCH FIRST 3 ROWS ONLY
)
)"""

sql = """SELECT CHUNK  from Vector_CV_PDF where (ID_PDF, id_chunk) in 
(
select ID_PDF ,substr(ranking,instr(ranking,'-',-1)+1)chunk_id from (
select ID_PDF , min(ranking) ranking
  from (
        select ID_PDF,VECTOR_DISTANCE(V,:1)||'--'||id_chunk ranking 
          from Vector_CV_PDF  
        order by VECTOR_DISTANCE(V,:2)
        )
group by id_pdf order by 2,1 desc FETCH FIRST 3 ROWS ONLY
)
)"""

with connection.cursor() as cursor:
        array_query1 = array.array("d", vector_prompt)
        array_query2 = array.array("d", vector_prompt)
        cursor.execute(sql,[array_query1,array_query1])
        rows = cursor.fetchall()
#print(rows)

AttributeError: 'str' object has no attribute 'embeddings'

Funzione cohere per generare testo

In [117]:
def generate_text(prompt, temp=0):
  response = co.generate(
    model='command',
    prompt=prompt,
    max_tokens=200,
    temperature=temp)
  return response.generations[0].text

Genera risposta

In [118]:

text_prompt ='Kindly answer the following question  ' + prompt_question  + ' based on  ' ;
for i in range(len(rows)):
    text_prompt += rows[i][0]+' and  '

#print (text_prompt)



response = generate_text(text_prompt, temp=0.5)
print(response)

Kindly answer the following question  I've opened a new position for an Italian coding expert in Taleo application , in the meantime could you providing internal name showing me the highlights of this person based on  first_name :  Paolo Last Name:Cecchi Professional Summary: Experienced Cloud Interconnect Specialist with a proven track record of designing and implementing robust cloud connectivity solutions. and  first_name :  Eugenio Last Name: Bernaci Personal Summary:born in 1970 and is grow up in rome. and  first_name :  Mario Last Name: Bianchi  Professional Summary: Dynamic and results-oriented Enterprise Technology Architect with a wealth of experience in crafting and implementing cutting-edge solutions within the Oracle ecosystem. and  


Your text contains a trailing whitespace, which has been trimmed to ensure high quality generations.


Based on the provided information, here are the internal names and role highlights for the new positions in Taleo application:

1. Paolo Cecchi - Italian Coding Expert
   - First name: Paolo
   - Last name: Cecchi
   - Professional Summary: Experienced Cloud Interconnect Specialist with a proven track record of designing and implementing robust cloud connectivity solutions in the Italian market. Paolo has extensive knowledge of network infrastructure and cloud computing, making him an ideal candidate to lead complex projects in the field of cloud interconnection.

2. Eugenio Bernaci - Italian Coding Expert
   - First name: Eugenio
   - Last name: Bernaci
   - Professional Summary: Seasoned Software Engineer with a strong background in cloud computing and software development. Eugenio's expertise in designing scalable and secure cloud architectures, as well as his proficiency in coding languages, positions him well to contribute to innovative projects within the Italian market.

3. Mari